# Merge ATAC-seq bed files 

In [1]:
tissues_with_atac = ['CHO','Brain','Liver','Lung','Kidney','Spleen','BMDMwt','BMDM1hKLA']

In [2]:
## Parameters specific to where your folders are and your data
parameter_file = 'parameters/params.yaml'
import yaml
import sys
import pandas as pd 
import os
with open(parameter_file,'r') as f:
    doc = yaml.load(f)

#p = dic2obj(**doc)

data_folder = doc['data_folder']
tissues = doc['tissues'].split(',')
sys.path.append(doc['pipeline_path'])
ref_fa = doc['ref_fa']
annotation=doc['annotation']
samples = pd.read_csv('parameters/samples.tsv',sep="\t", index_col=0)

homer_path =  "/data2/resources/software/homer/v4.11_10242019/bin/"
os.environ['PATH'] = f"{homer_path}:{os.environ['PATH']}"
!which homer


/data2/resources/software/homer/v4.11_10242019/bin//homer


/home/isshamie/software/anaconda2/envs/TSS/lib/python3.7/site-packages/ipykernel_launcher.py:8: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  


In [3]:
import os
import sys
import pandas as pd
import matplotlib
import seaborn as sns
import pickle
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from itertools import product
import glob
import re
from matplotlib_venn import venn2
from matplotlib import rcParams
import inspect
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
sys.setrecursionlimit(3000)
%load_ext autoreload
%autoreload 2
rcParams['figure.figsize'] = 8, 6
import tqdm
### notebook specific configuration ###
from os.path import basename
#mpl.style.use('ggplot')
mpl.style.use('fivethirtyeight')
from cycler import cycler
mpl.rcParams['axes.prop_cycle'] = cycler(color='bgrcmyk')

from Homer import *

print('Number of tissues: ',len(tissues))

Number of tissues:  13


## Copy peaks to folder to sort

In [4]:
samples

,Experiment,Full,Name,Tissue
CHBMDM1hKLA2_ATAC_JHS1000_SD_AGGTTGGG_S11_L001_R1_001.fastq.gz,ATAC,CHBMDM1hKLA2_ATAC_JHS1000_SD_AGGTTGGG_S11_L001...,BMDM1hKLA_ATAC1,BMDM1hKLA
CHBMDM1hKLA1_ATAC_JHS998_SD_TGGGTTTC_S9_L001_R1_001.fastq.gz,ATAC,CHBMDM1hKLA1_ATAC_JHS998_SD_TGGGTTTC_S9_L001_R...,BMDM1hKLA_ATAC2,BMDM1hKLA
CHBMDM1hKLA_GRO_JHS1027_SD_CAAAAG_S42_L003_R1_001.fastq.gz,GRO,CHBMDM1hKLA_GRO_JHS1027_SD_CAAAAG_S42_L003_R1_...,BMDM1hKLA_GRO1,BMDM1hKLA
CHBMDM1hKLA_GRO_JHS1034_SD_AGTCAA_S49_L003_R1_001.fastq.gz,GROCap,CHBMDM1hKLA_GRO_JHS1034_SD_AGTCAA_S49_L003_R1_...,BMDM1hKLA_GROCap1,BMDM1hKLA
CHBMDMVehic2_ATAC_JHS999_SD_TGGTCACA_S10_L001_R1_001.fastq.gz,ATAC,CHBMDMVehic2_ATAC_JHS999_SD_TGGTCACA_S10_L001_...,BMDMwt_ATAC1,BMDMwt
...,...,...,...,...
NL5_Liver_S5_L006_R1_001.fastq.gz,RNASeq,NL5_Liver_S5_L006_R1_001.fastq.gz,Liver_RNASeq1,Liver
NL6_Heart_S6_L006_R1_001.fastq.gz,RNASeq,NL6_Heart_S6_L006_R1_001.fastq.gz,Heart_RNASeq1,Heart
NL7_Brain2_S7_L006_R1_001.fastq.gz,RNASeq,NL7_Brain2_S7_L006_R1_001.fastq.gz,Brain_RNASeq2,Brain
NL8_Muscle_S8_L006_R1_001.fastq.gz,RNASeq,NL8_Muscle_S8_L006_R1_001.fastq.gz,Muscle_RNASeq1,Muscle


In [5]:
tissues_with_atac = (samples[samples["Experiment"] == "ATAC"]['Tissue'].unique())


In [6]:
os.chdir(doc["results"])
atac_out = "ATAC_Results/optimal_set"
if not os.path.exists(atac_out):
    os.makedirs(atac_out)

In [7]:
data_folder

'/data/isshamie/TSS/NCBI_PICR_103_Processed/'

In [8]:
## Sort ATAC bed files
for t in tissues_with_atac:
    print(t)
    atac_f = glob.glob(os.path.join(data_folder, t, 'ATAC/out/peak/macs2/overlap/conservative_set/*narrowPeak.gz'))[0]
    atac_f_out = atac_f.replace('.gz','') + '.sort'
    atac_f_out = os.path.join(atac_out,t,os.path.basename(atac_f).replace('.gz','.sort'))
    if not os.path.exists(os.path.join(atac_out,t)):
        os.mkdir(os.path.join(atac_out,t))
    
    cmd = 'bedtools sort -i %s > %s' % (atac_f,atac_f_out)   
    print(cmd)
    os.system(cmd)

BMDM1hKLA
bedtools sort -i /data/isshamie/TSS/NCBI_PICR_103_Processed/BMDM1hKLA/ATAC/out/peak/macs2/overlap/conservative_set/ATAC_rep1-rep2.naive_overlap.narrowPeak.gz > ATAC_Results/optimal_set/BMDM1hKLA/ATAC_rep1-rep2.naive_overlap.narrowPeak.sort
BMDMwt
bedtools sort -i /data/isshamie/TSS/NCBI_PICR_103_Processed/BMDMwt/ATAC/out/peak/macs2/overlap/conservative_set/ATAC_rep1-rep2.naive_overlap.narrowPeak.gz > ATAC_Results/optimal_set/BMDMwt/ATAC_rep1-rep2.naive_overlap.narrowPeak.sort
Brain
bedtools sort -i /data/isshamie/TSS/NCBI_PICR_103_Processed/Brain/ATAC/out/peak/macs2/overlap/conservative_set/ATAC_rep1-pr.naive_overlap.narrowPeak.gz > ATAC_Results/optimal_set/Brain/ATAC_rep1-pr.naive_overlap.narrowPeak.sort
CHO
bedtools sort -i /data/isshamie/TSS/NCBI_PICR_103_Processed/CHO/ATAC/out/peak/macs2/overlap/conservative_set/ATAC_rep1-rep2.naive_overlap.narrowPeak.gz > ATAC_Results/optimal_set/CHO/ATAC_rep1-rep2.naive_overlap.narrowPeak.sort
Kidney
bedtools sort -i /data/isshamie/TSS/

In [9]:
all_bed = []
for t in tissues_with_atac:
    all_bed = all_bed + glob.glob(os.path.join(atac_out,t,"*sort"))

cmd = f'mergePeaks {" ".join(all_bed)} > {atac_out}/ATAC_merge.merge'
#cmd = "bedtools merge -i " + " ".join(all_bed) + " > Results/ATAC_results/ATAC_merge.bed"
#!{cmd}
print(cmd)
!{cmd}

mergePeaks ATAC_Results/optimal_set/BMDM1hKLA/ATAC_rep1-rep2.naive_overlap.narrowPeak.sort ATAC_Results/optimal_set/BMDMwt/ATAC_rep1-rep2.naive_overlap.narrowPeak.sort ATAC_Results/optimal_set/Brain/ATAC_rep1-pr.naive_overlap.narrowPeak.sort ATAC_Results/optimal_set/CHO/ATAC_rep1-rep2.naive_overlap.narrowPeak.sort ATAC_Results/optimal_set/Kidney/ATAC_rep1-pr.naive_overlap.narrowPeak.sort ATAC_Results/optimal_set/Liver/ATAC_rep1-pr.naive_overlap.narrowPeak.sort ATAC_Results/optimal_set/Lung/ATAC_rep1-pr.naive_overlap.narrowPeak.sort ATAC_Results/optimal_set/Spleen/ATAC_rep1-pr.naive_overlap.narrowPeak.sort > ATAC_Results/optimal_set/ATAC_merge.merge
	Max distance to merge: direct overlap required (-d given)
	Merging peaks... 
	Comparing ATAC_Results/optimal_set/BMDM1hKLA/ATAC_rep1-rep2.naive_overlap.narrowPeak.sort (113031 total) and ATAC_Results/optimal_set/BMDM1hKLA/ATAC_rep1-rep2.naive_overlap.narrowPeak.sort (113031 total)
	Comparing ATAC_Results/optimal_set/BMDM1hKLA/ATAC_rep1-rep2

	Comparing ATAC_Results/optimal_set/Kidney/ATAC_rep1-pr.naive_overlap.narrowPeak.sort (110673 total) and ATAC_Results/optimal_set/Spleen/ATAC_rep1-pr.naive_overlap.narrowPeak.sort (43775 total)
	Comparing ATAC_Results/optimal_set/Liver/ATAC_rep1-pr.naive_overlap.narrowPeak.sort (15913 total) and ATAC_Results/optimal_set/BMDM1hKLA/ATAC_rep1-rep2.naive_overlap.narrowPeak.sort (113031 total)
	Comparing ATAC_Results/optimal_set/Liver/ATAC_rep1-pr.naive_overlap.narrowPeak.sort (15913 total) and ATAC_Results/optimal_set/BMDMwt/ATAC_rep1-rep2.naive_overlap.narrowPeak.sort (109721 total)
	Comparing ATAC_Results/optimal_set/Liver/ATAC_rep1-pr.naive_overlap.narrowPeak.sort (15913 total) and ATAC_Results/optimal_set/Brain/ATAC_rep1-pr.naive_overlap.narrowPeak.sort (107441 total)
	Comparing ATAC_Results/optimal_set/Liver/ATAC_rep1-pr.naive_overlap.narrowPeak.sort (15913 total) and ATAC_Results/optimal_set/CHO/ATAC_rep1-rep2.naive_overlap.narrowPeak.sort (83795 total)
	Comparing ATAC_Results/optima

X	X		X					5078	ATAC_Results/optimal_set/BMDM1hKLA/ATAC_rep1-rep2.naive_overlap.narrowPeak.sort|ATAC_Results/optimal_set/BMDMwt/ATAC_rep1-rep2.naive_overlap.narrowPeak.sort|ATAC_Results/optimal_set/CHO/ATAC_rep1-rep2.naive_overlap.narrowPeak.sort
X	X		X				X	316	ATAC_Results/optimal_set/BMDM1hKLA/ATAC_rep1-rep2.naive_overlap.narrowPeak.sort|ATAC_Results/optimal_set/BMDMwt/ATAC_rep1-rep2.naive_overlap.narrowPeak.sort|ATAC_Results/optimal_set/CHO/ATAC_rep1-rep2.naive_overlap.narrowPeak.sort|ATAC_Results/optimal_set/Spleen/ATAC_rep1-pr.naive_overlap.narrowPeak.sort
X	X		X			X		329	ATAC_Results/optimal_set/BMDM1hKLA/ATAC_rep1-rep2.naive_overlap.narrowPeak.sort|ATAC_Results/optimal_set/BMDMwt/ATAC_rep1-rep2.naive_overlap.narrowPeak.sort|ATAC_Results/optimal_set/CHO/ATAC_rep1-rep2.naive_overlap.narrowPeak.sort|ATAC_Results/optimal_set/Lung/ATAC_rep1-pr.naive_overlap.narrowPeak.sort
X	X		X			X	X	181	ATAC_Results/optimal_set/BMDM1hKLA/ATAC_rep1-rep2.naive_overlap.narrowPeak.sort|ATAC_Results/o

X	X	X		X				1718	ATAC_Results/optimal_set/BMDM1hKLA/ATAC_rep1-rep2.naive_overlap.narrowPeak.sort|ATAC_Results/optimal_set/BMDMwt/ATAC_rep1-rep2.naive_overlap.narrowPeak.sort|ATAC_Results/optimal_set/Brain/ATAC_rep1-pr.naive_overlap.narrowPeak.sort|ATAC_Results/optimal_set/Kidney/ATAC_rep1-pr.naive_overlap.narrowPeak.sort
X	X	X		X			X	531	ATAC_Results/optimal_set/BMDM1hKLA/ATAC_rep1-rep2.naive_overlap.narrowPeak.sort|ATAC_Results/optimal_set/BMDMwt/ATAC_rep1-rep2.naive_overlap.narrowPeak.sort|ATAC_Results/optimal_set/Brain/ATAC_rep1-pr.naive_overlap.narrowPeak.sort|ATAC_Results/optimal_set/Kidney/ATAC_rep1-pr.naive_overlap.narrowPeak.sort|ATAC_Results/optimal_set/Spleen/ATAC_rep1-pr.naive_overlap.narrowPeak.sort
X	X	X		X		X		1267	ATAC_Results/optimal_set/BMDM1hKLA/ATAC_rep1-rep2.naive_overlap.narrowPeak.sort|ATAC_Results/optimal_set/BMDMwt/ATAC_rep1-rep2.naive_overlap.narrowPeak.sort|ATAC_Results/optimal_set/Brain/ATAC_rep1-pr.naive_overlap.narrowPeak.sort|ATAC_Results/optimal_set/Kidne

		X		X				6569	ATAC_Results/optimal_set/Brain/ATAC_rep1-pr.naive_overlap.narrowPeak.sort|ATAC_Results/optimal_set/Kidney/ATAC_rep1-pr.naive_overlap.narrowPeak.sort
		X		X			X	299	ATAC_Results/optimal_set/Brain/ATAC_rep1-pr.naive_overlap.narrowPeak.sort|ATAC_Results/optimal_set/Kidney/ATAC_rep1-pr.naive_overlap.narrowPeak.sort|ATAC_Results/optimal_set/Spleen/ATAC_rep1-pr.naive_overlap.narrowPeak.sort
		X		X		X		924	ATAC_Results/optimal_set/Brain/ATAC_rep1-pr.naive_overlap.narrowPeak.sort|ATAC_Results/optimal_set/Kidney/ATAC_rep1-pr.naive_overlap.narrowPeak.sort|ATAC_Results/optimal_set/Lung/ATAC_rep1-pr.naive_overlap.narrowPeak.sort
		X		X		X	X	135	ATAC_Results/optimal_set/Brain/ATAC_rep1-pr.naive_overlap.narrowPeak.sort|ATAC_Results/optimal_set/Kidney/ATAC_rep1-pr.naive_overlap.narrowPeak.sort|ATAC_Results/optimal_set/Lung/ATAC_rep1-pr.naive_overlap.narrowPeak.sort|ATAC_Results/optimal_set/Spleen/ATAC_rep1-pr.naive_overlap.narrowPeak.sort
X			X	X	X			6	ATAC_Results/optimal_set/BMDM1hKL

X	X							28266	ATAC_Results/optimal_set/BMDM1hKLA/ATAC_rep1-rep2.naive_overlap.narrowPeak.sort|ATAC_Results/optimal_set/BMDMwt/ATAC_rep1-rep2.naive_overlap.narrowPeak.sort
X	X						X	1849	ATAC_Results/optimal_set/BMDM1hKLA/ATAC_rep1-rep2.naive_overlap.narrowPeak.sort|ATAC_Results/optimal_set/BMDMwt/ATAC_rep1-rep2.naive_overlap.narrowPeak.sort|ATAC_Results/optimal_set/Spleen/ATAC_rep1-pr.naive_overlap.narrowPeak.sort
X	X					X		1207	ATAC_Results/optimal_set/BMDM1hKLA/ATAC_rep1-rep2.naive_overlap.narrowPeak.sort|ATAC_Results/optimal_set/BMDMwt/ATAC_rep1-rep2.naive_overlap.narrowPeak.sort|ATAC_Results/optimal_set/Lung/ATAC_rep1-pr.naive_overlap.narrowPeak.sort
X	X					X	X	828	ATAC_Results/optimal_set/BMDM1hKLA/ATAC_rep1-rep2.naive_overlap.narrowPeak.sort|ATAC_Results/optimal_set/BMDMwt/ATAC_rep1-rep2.naive_overlap.narrowPeak.sort|ATAC_Results/optimal_set/Lung/ATAC_rep1-pr.naive_overlap.narrowPeak.sort|ATAC_Results/optimal_set/Spleen/ATAC_rep1-pr.naive_overlap.narrowPeak.sort
X			X					22

X	X	X	X	X			X	1015	ATAC_Results/optimal_set/BMDM1hKLA/ATAC_rep1-rep2.naive_overlap.narrowPeak.sort|ATAC_Results/optimal_set/BMDMwt/ATAC_rep1-rep2.naive_overlap.narrowPeak.sort|ATAC_Results/optimal_set/Brain/ATAC_rep1-pr.naive_overlap.narrowPeak.sort|ATAC_Results/optimal_set/CHO/ATAC_rep1-rep2.naive_overlap.narrowPeak.sort|ATAC_Results/optimal_set/Kidney/ATAC_rep1-pr.naive_overlap.narrowPeak.sort|ATAC_Results/optimal_set/Spleen/ATAC_rep1-pr.naive_overlap.narrowPeak.sort
X	X	X	X	X		X		2012	ATAC_Results/optimal_set/BMDM1hKLA/ATAC_rep1-rep2.naive_overlap.narrowPeak.sort|ATAC_Results/optimal_set/BMDMwt/ATAC_rep1-rep2.naive_overlap.narrowPeak.sort|ATAC_Results/optimal_set/Brain/ATAC_rep1-pr.naive_overlap.narrowPeak.sort|ATAC_Results/optimal_set/CHO/ATAC_rep1-rep2.naive_overlap.narrowPeak.sort|ATAC_Results/optimal_set/Kidney/ATAC_rep1-pr.naive_overlap.narrowPeak.sort|ATAC_Results/optimal_set/Lung/ATAC_rep1-pr.naive_overlap.narrowPeak.sort
X	X	X	X	X		X	X	6768	ATAC_Results/optimal_set/BMDM1hKLA